# Chapter 1: AC Conductivity and Permittivity

#### The Drude Model allows one to make prediction about measurable material properties. This jupyter notebook utilises the concepts from the lecture to calculate plots for the AC conductivity and permittivity of a material. These plots allow one to study the dependency of the aforementioned properties on DC resistivity and electron density of the material.

In [1]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

Loading BokehJS ...

#### First, we set up variables used in the calculation. These involve defining electron density, electron mass and charge, material resistivity/conductivity, vacuum and effective material permittivity, the bounds of frequency used in calculations and the number of points we want to calculate for our graphs. Additionally, mean free time tau can be calculated from 
$\sigma = ne^2\tau /m $

In [2]:
# Set up constant in SI units
dens = 8.47e+28 #Define initial electron density
m = 9.109e-31 #Define electron mass
elec = 1.602e-19 #Define elementary electron charge
rho = 1.56e-8 #Define DC resistivity
sigma0 = 1/rho #Calculate initial DC conductivity from resisitivity
eps = 8.854e-12 #Vacuum permittivity
eps0 = 1.0 #Effective permittivity of the material
freq_min = 1e-12   #Lower limit of frequency (approximately DC scenario)
freq_max = 5000.0    #Upper limit of frequency
N = 5000   #Number of points for the independent variable
tau = m / (dens*rho*elec**2) #Calculate mean free time

#### Next, we define a variable containing all our frequency points and convert frequecy to angular frequency. With all variables defined, we can calculate the real and imaginary part of the AC conductivity.

In [3]:
# Set up the variable space
freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
w = 2*np.pi*freq*1e+12 #Convert frequency to angular frequency
# Calculate AC conductivity
sigma1 = sigma0 / (1.0 + w**2*tau**2)  #Real part of the conductivity
sigma2 = w*tau*sigma0 / (1.0 + w**2*tau**2)  #imaginary part of the conductivity

#### With AC coductivity calculated, we can proceed to calculate the permittivity of the material.

In [4]:
#calculate permittivity
eps1 = eps0 - sigma2 /(eps*w)
eps2 = sigma1/(eps*w)

#### In the two inputs below we set up the plot figures four our 4 graphs (imaginary and real part of conductivity, imaginary and real part of permittivity)

In [5]:
# Set up conductivity plot
plot = figure(plot_height=200, plot_width=400, title="Real part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              y_axis_type="log", x_range=[freq_min, freq_max], y_range=[1e+4, 8e+7])
plot.xaxis[0].axis_label='Frequency (THz)'
plot.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
r = plot.line(freq, sigma1, line_width=3, line_alpha=0.6)

plot2 = figure(plot_height=200, plot_width=400, title="Imaginary part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max], y_range=[0.0, 2e+7])
plot2.xaxis[0].axis_label='Frequency (THz)'
plot2.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
rr = plot2.line(freq, sigma2, line_width=3, line_alpha=0.6, color = 'red')

In [6]:
# Set up permittivity plot
plot3 = figure(plot_height=200, plot_width=400, title="Real part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_axis_type="log", x_range=[freq_min, freq_max], y_range=[-6e+6, 5e+4])
plot3.xaxis[0].axis_label='Frequency (THz)'
plot3.yaxis[0].axis_label='Permittivity (F/m)'
rrr = plot3.line(freq, eps1, line_width=3, line_alpha=0.6)

plot4 = figure(plot_height=200, plot_width=400, title= "imaginary part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_axis_type="log", x_range=[freq_min, freq_max], y_range=[0, 6.5e+20])
plot4.xaxis[0].axis_label='Frequency (THz)'
plot4.yaxis[0].axis_label='Permittivity (F/m)'
rrrr = plot4.line(freq, eps2, line_width=3, line_alpha=0.6, color = 'red')

#### As final step, we define an update function which would allow us to use sliders to change input variables (such as density and DC resistivity) and see the change in the plot without having to rerun the code.

In [7]:
# Set up callbacks to update the plots
def update_data(density, resistivity):
# Generate the new curve
    dens = density*1e+28
    rho = resistivity*1e-8
    sigma0 = 1/rho
    tau = m / (dens*rho*elec**2)
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq*1e+9 #Convert frequency to angular frequency
    r.data_source.data['y'] = sigma0 / (1.0 + w**2*tau**2) 
    rr.data_source.data['y'] = w*tau*sigma0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)

    push_notebook()

#### We can now plot the data

In [8]:
show(column([row(plot,plot2),row(plot3, plot4)]),notebook_handle=True)
interact(update_data, density = (0.091, 24.7, 0.02), resistivity = (2.8, 107, 0.5))

interactive(children=(FloatSlider(value=12.391, description='density', max=24.7, min=0.091, step=0.02), FloatS…

<function __main__.update_data(density, resistivity)>